In [91]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = sorted(glob.glob('../features/*.pkl'))
for l in fl:
    print("'{}',".format(l))

'../features/cnn_1_aug1_feat.pkl',
'../features/cnn_1_aug_1_fold8_feat.pkl',
'../features/cnn_1_aug_rescale_preprocess_feat.pkl',
'../features/cnn_1_aug_skimage_denoise_feat.pkl',
'../features/cnn_1_aug_skimage_preprocess_feat.pkl',
'../features/cnn_1_feat.pkl',
'../features/cnn_2_aug1_feat.pkl',
'../features/cnn_2_aug_1_fold8_feat.pkl',
'../features/cnn_2_aug_denoise_preprocess_feat.pkl',
'../features/cnn_2_aug_rescale_preprocess_feat.pkl',
'../features/cnn_2_aug_skimage_preprocess_feat.pkl',
'../features/cnn_2_feat.pkl',
'../features/cnn_3_aug1_feat.pkl',
'../features/cnn_3_aug_1_fold8_feat.pkl',
'../features/cnn_3_aug_denoise_preprocess_feat.pkl',
'../features/cnn_3_aug_rescale_preprocess_feat.pkl',
'../features/cnn_3_aug_skimage_preprocess_feat.pkl',
'../features/cnn_4_aug1_feat.pkl',
'../features/cnn_4_aug1_feat_add_early.pkl',
'../features/cnn_4_aug_1_fold8_feat.pkl',
'../features/cnn_4_aug_denoise_preprocess_feat.pkl',
'../features/cnn_4_aug_rescale_preprocess_feat.pkl',
'../fea

In [92]:
choose_fl = [
'../features/cnn_1_aug1_feat.pkl',
'../features/cnn_1_aug_1_fold8_feat.pkl',
'../features/cnn_1_aug_rescale_preprocess_feat.pkl',
'../features/cnn_1_aug_skimage_denoise_feat.pkl',
'../features/cnn_1_aug_skimage_preprocess_feat.pkl',
'../features/cnn_1_feat.pkl',
'../features/cnn_2_aug1_feat.pkl',
#'../features/cnn_2_aug_1_fold8_feat.pkl',
'../features/cnn_2_aug_denoise_preprocess_feat.pkl',
'../features/cnn_2_aug_rescale_preprocess_feat.pkl',
'../features/cnn_2_aug_skimage_preprocess_feat.pkl',
'../features/cnn_2_feat.pkl',
'../features/cnn_3_aug1_feat.pkl',
'../features/cnn_3_aug_1_fold8_feat.pkl',
'../features/cnn_3_aug_denoise_preprocess_feat.pkl',
'../features/cnn_3_aug_rescale_preprocess_feat.pkl',
'../features/cnn_3_aug_skimage_preprocess_feat.pkl',
'../features/cnn_4_aug1_feat.pkl',
'../features/cnn_4_aug1_feat_add_early.pkl',
'../features/cnn_4_aug_1_fold8_feat.pkl',
'../features/cnn_4_aug_denoise_preprocess_feat.pkl',
'../features/cnn_4_aug_rescale_preprocess_feat.pkl',
'../features/cnn_4_aug_skimage_preprocess_feat.pkl',
'../features/cnn_5_aug1_feat.pkl',
'../features/cnn_5_aug_1_fold8_feat.pkl',
'../features/cnn_5_aug_denoise_preprocess_feat.pkl',
'../features/cnn_5_aug_rescale_preprocess_feat.pkl',
'../features/cnn_5_aug_skimage_preprocess_feat.pkl',
'../features/incept_1_feat.pkl',
'../features/incept_aug1_feat.pkl',
'../features/incept_aug1_fold8_feat.pkl',
'../features/incept_aug2_feat.pkl',
'../features/incept_aug3_feat.pkl',
'../features/incept_aug4_feat.pkl',
'../features/other_feat.pkl',
'../features/other_model_gbrt.pkl',
'../features/other_model_gbrt2.pkl',
'../features/other_model_lgb.pkl',
'../features/other_model_lgb2.pkl',
'../features/other_model_lr.pkl',
'../features/other_model_lr2.pkl',
'../features/other_model_rf.pkl',
'../features/other_model_rf2.pkl',
'../features/other_model_xgb.pkl',
'../features/other_model_xgb2.pkl',
'../features/resnet_1_feat.pkl',
'../features/resnet_aug1_feat.pkl',
'../features/resnet_aug1_fold8_feat.pkl',
'../features/resnet_aug2_feat.pkl',
'../features/resnet_aug3_feat.pkl',
'../features/resnet_aug4_feat.pkl',
'../features/vgg16_1_feat.pkl',
'../features/vgg_aug1_feat.pkl',
'../features/vgg_aug1_fold8_feat.pkl',
'../features/vgg_aug1_new_feat.pkl',
'../features/xception_aug1_feat.pkl',

]

def load_x_y():
    train_x,test_x=[],[]
    for f in choose_fl:
        print(f)
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

../features/cnn_1_aug1_feat.pkl
../features/cnn_1_aug_1_fold8_feat.pkl
../features/cnn_1_aug_rescale_preprocess_feat.pkl
../features/cnn_1_aug_skimage_denoise_feat.pkl
../features/cnn_1_aug_skimage_preprocess_feat.pkl
../features/cnn_1_feat.pkl
../features/cnn_2_aug1_feat.pkl
../features/cnn_2_aug_denoise_preprocess_feat.pkl
../features/cnn_2_aug_rescale_preprocess_feat.pkl
../features/cnn_2_aug_skimage_preprocess_feat.pkl
../features/cnn_2_feat.pkl
../features/cnn_3_aug1_feat.pkl
../features/cnn_3_aug_1_fold8_feat.pkl
../features/cnn_3_aug_denoise_preprocess_feat.pkl
../features/cnn_3_aug_rescale_preprocess_feat.pkl
../features/cnn_3_aug_skimage_preprocess_feat.pkl
../features/cnn_4_aug1_feat.pkl
../features/cnn_4_aug1_feat_add_early.pkl
../features/cnn_4_aug_1_fold8_feat.pkl
../features/cnn_4_aug_denoise_preprocess_feat.pkl
../features/cnn_4_aug_rescale_preprocess_feat.pkl
../features/cnn_4_aug_skimage_preprocess_feat.pkl
../features/cnn_5_aug1_feat.pkl
../features/cnn_5_aug_1_fold8_

In [93]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        params = {
                'colsample_bytree': 0.75,
                'colsample_bylevel':0.95,
                'gamma':0.2,
                'subsample': 0.9,
                'eta': 0.07,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight': 0.85
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(test_x)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 500, watchlist, 
                        early_stopping_rounds=20,
                        verbose_eval=1009)
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict(d_valid))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            w_test_pred = m.predict(d_test)*curr_r_val_loss
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred
        print(submission.head())
        submission.to_csv('../results/xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=w_test_pred
        print(submission.head())
        submission.to_csv('../results/weighted_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

    
print('def done')

def done


In [94]:
cv_test(3,42,False)

[0]	train-logloss:0.636051	valid-logloss:0.639897
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[117]	train-logloss:0.046343	valid-logloss:0.149628

[0]	train-logloss:0.636598	valid-logloss:0.639232
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[92]	train-logloss:0.063647	valid-logloss:0.157304

[0]	train-logloss:0.636864	valid-logloss:0.638841
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[70]	train-logloss:0.076399	valid-logloss:0.158341

local average valid loss 0.157166433634 val loss std 0.00349712555524


In [95]:
#cv_test(5,42,True)

In [96]:
#cv_test(10,42,True)

In [97]:
#cv_test(3,233,True)

In [98]:
#cv_test(5,233,True)

In [99]:
#cv_test(10,233,True)